<h1 style="color: #007bff;"><center>Impact of COVID-19 & Overdoses on Pharmaceutical Stocks and ETFs</center></h1>
<ul>
<li>Group Number: 21</li>
<li>Student: Karston Kuciemba</li>
<li>UT EID: ktk654</li>
<li>Student: Ananya Kaalva</li>
<li>UT EID: ak46562</li>
<li>Student: Carolyn Pyun</li>
<li>UT EID: cgp755</li>
<li>Date:  3/16/2024</li>
</ul>

In [127]:
# import needed libraries
import numpy as np
import pandas as pd
import math
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns

<h3 style="padding-left: 30px; color: #007bff;">Read in dataframes for visualization</h3>

In [134]:
# covid dfs
tickers = ['LLY', 'JNJ', 'MRK', 'ABBV', 'MRNA', 'PFE', 'AMGN', 'PPH', 'IHE', 'PJP']

covid_dfs= {}

for symbol in tickers:
    # for each symbol, load the pkl file and store it in the dictionary as a df
    covid_dfs[f"{symbol}_df-covid"] = pd.read_csv(f"../pharma-data/merged-dfs/{symbol}_df-covid.csv", parse_dates=True)

# remove last 182 lines bc it is 0 covid deaths (stopped reporting)
for symbol in tickers:
    covid_dfs[f"{symbol}_df-covid"] = covid_dfs[f"{symbol}_df-covid"].iloc[:-182]

for symbol in tickers:
    covid_dfs[f"{symbol}_df-covid"] = covid_dfs[f"{symbol}_df-covid"].rename(columns={"Unnamed: 0": "Date"})

In [135]:
# overdose dfs
tickers = ['LLY', 'JNJ', 'MRK', 'ABBV', 'MRNA', 'PFE', 'AMGN', 'PPH', 'IHE', 'PJP']

overdose_dfs= {}

for symbol in tickers:
    # for each symbol, load the pkl file and store it in the dictionary as a df
    overdose_dfs[f"{symbol}_df-overdose"] = pd.read_csv(f"../pharma-data/merged-dfs/{symbol}_df-overdose.csv", parse_dates=True)

for symbol in tickers:
    overdose_dfs[f"{symbol}_df-overdose"] = overdose_dfs[f"{symbol}_df-overdose"].rename(columns={"Unnamed: 0": "Date"})

# <center style="text-align: center; color: #007bff;">Visualization 1</center>

<h3><center style="color: #DE3163;"><strong>Goal of Visual:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
       The purpose of this visualization is to show the variance in pharmaceutical stock prices as it relates to COVID-19. How large were the discrepencies between COVID-19 treatment and/or prevention-related companies vs. non-related ones?
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visualization Tools:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Altair (Bar Chart & Lines)
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Argument Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Some companies performed much greater than others. Is this just noise? While the market is unpredictable and no one can know for certain, there is an
        extent of confidence that can be attributed to the idea that any entity with the possibility of treating COVID-19.
    </li>
    <li style="font-size: 16px;">
        These companies performed "better": LLY (332%), MRNA (702%), 
    </li>
    <li style="font-size: 16px;">
        Our analysis found the statistical significance here: LIST HERE
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visual Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe the visual that goes with this argument
    </li>
    <li style="font-size: 16px;">
        Describe the interactivity
    </li>
    <li style="font-size: 16px;">
        Describe the layout, choice of colors, annotations (if any), and legends (if any)
    </li>
</ul>

In [137]:
stock_lines = None

# loop through each df in covid_dfs. for each one, plot their prices over time
for symbol in tickers:
    # create a line for each symbol with a unique color
    stock_line = alt.Chart(covid_dfs[f"{symbol}_df-covid"]).mark_line().encode(
        x=alt.X('Date:T', axis=alt.Axis(format='%m/%Y')),
        y=alt.Y('Close:Q').scale(type='log'),
        color=alt.Color('symbol:N', scale=alt.Scale(scheme='category20')),
        tooltip=[alt.Tooltip('symbol:N', title='Ticker'), alt.Tooltip('Date:T', title='Date'), alt.Tooltip('Close:Q', title='Closing Price'), alt.Tooltip('US covid deaths:Q', title='US COVID Deaths')]
    ).properties(
        title="Stock Prices during COVID",
        width=1000,
        height=450
    ).transform_calculate(
        symbol=f"'{symbol}'"  # pass the symbol value into the chart
    ).interactive()
    
    # place each new line chart on combined chart
    if stock_lines is None:
        stock_lines = stock_line
    else:
        stock_lines += stock_line

# create area chart for covid deaths (choose random df because they all have the same covid deaths)
covid_deaths_area = alt.Chart(covid_dfs['LLY_df-covid']).mark_area(color='black', opacity=0.15).encode(
    x=alt.X('Date:T', axis=alt.Axis(format='%m/%Y')),
    y=alt.Y('US covid deaths:Q'),
)

final_chart = alt.layer(covid_deaths_area, stock_lines).resolve_scale(y='independent')

final_chart


KeyError: 'NVO_df-covid'

# <center style="text-align: center; color: #007bff;">Visualization 2</center>

<h3><center style="color: #DE3163;"><strong>Goal of Visual:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visualization Tools:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Argument Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Present in a bulleted form your argument (add "li" items for each bullet)
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visual Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe the visual that goes with this argument
    </li>
    <li style="font-size: 16px;">
        Describe the interactivity
    </li>
    <li style="font-size: 16px;">
        Describe the layout, choice of colors, annotations (if any), and legends (if any)
    </li>
</ul>

# <center style="text-align: center; color: #007bff;">Visualization 3</center>

<h3><center style="color: #DE3163;"><strong>Goal of Visual:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visualization Tools:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Argument Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Present in a bulleted form your argument (add "li" items for each bullet)
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visual Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe the visual that goes with this argument
    </li>
    <li style="font-size: 16px;">
        Describe the interactivity
    </li>
    <li style="font-size: 16px;">
        Describe the layout, choice of colors, annotations (if any), and legends (if any)
    </li>
</ul>

# <center style="text-align: center; color: #007bff;">Visualization 4</center>

<h3><center style="color: #DE3163;"><strong>Goal of Visual:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visualization Tools:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Argument Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Present in a bulleted form your argument (add "li" items for each bullet)
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visual Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe the visual that goes with this argument
    </li>
    <li style="font-size: 16px;">
        Describe the interactivity
    </li>
    <li style="font-size: 16px;">
        Describe the layout, choice of colors, annotations (if any), and legends (if any)
    </li>
</ul>

# <center style="text-align: center; color: #007bff;">Visualization 5</center>

<h3><center style="color: #DE3163;"><strong>Goal of Visual:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visualization Tools:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Argument Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Present in a bulleted form your argument (add "li" items for each bullet)
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visual Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe the visual that goes with this argument
    </li>
    <li style="font-size: 16px;">
        Describe the interactivity
    </li>
    <li style="font-size: 16px;">
        Describe the layout, choice of colors, annotations (if any), and legends (if any)
    </li>
</ul>